In [1]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator



In [2]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)



In [3]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:

# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'])



In [5]:
# Train the neural network/model
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion-model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('emotion-model.h5')



Epoch 1/50
448/448 [==============================] - 254s 562ms/step - loss: 1.7903 - accuracy: 0.2661 - val_loss: 1.6797 - val_accuracy: 0.3531
Epoch 2/50
448/448 [==============================] - 249s 557ms/step - loss: 1.6150 - accuracy: 0.3743 - val_loss: 1.5321 - val_accuracy: 0.4109
Epoch 3/50
448/448 [==============================] - 252s 562ms/step - loss: 1.5105 - accuracy: 0.4185 - val_loss: 1.4506 - val_accuracy: 0.4449
Epoch 4/50
448/448 [==============================] - 252s 563ms/step - loss: 1.4363 - accuracy: 0.4500 - val_loss: 1.3798 - val_accuracy: 0.4770
Epoch 5/50
448/448 [==============================] - 253s 564ms/step - loss: 1.3733 - accuracy: 0.4757 - val_loss: 1.3266 - val_accuracy: 0.4954
Epoch 6/50
448/448 [==============================] - 253s 565ms/step - loss: 1.3210 - accuracy: 0.4978 - val_loss: 1.2894 - val_accuracy: 0.5059
Epoch 7/50
448/448 [==============================] - 255s 569ms/step - loss: 1.2779 - accuracy: 0.5149 - val_loss: 1.2584 -

In [10]:
# Save the entire model to a single HDF5 file
emotion_model.save('emotion-model-complete.h5')